In [1]:
import pandas as pd
import pydot
import numpy as np
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential, Model
from keras.layers import Dense,Input, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras import losses
from keras.constraints import maxnorm
from keras.models import load_model
from keras.utils import plot_model
from keras.layers.merge import concatenate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
from commons import mean_absolute_percentage_error
from IPython.display import Audio
sound_file = 'beep.wav'

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [3]:
# load dataset
dataframe = pd.read_csv("notebooks/datasets/reg_seven.csv", sep=',')
dataframe.shape

(2466, 14)

In [4]:
dataframe.head(3)

,difficulty30rsi,difficulty90var,fee_to_reward90rsiUSD,hashrate30var,median_transaction_fee7rocUSD,mining_profitability,price30smaUSD,price3wmaUSD,price7wmaUSD,sentinusd90emaUSD,size90trx,top100cap,transactionvalueUSD,priceUSD
0,97.889,1.168546e+12,49.775,9.266842e+25,36.980,7220.0,56.481,91.616,88.156,68391477,0.334,19.962,2592.0,94.715
1,97.889,1.243755e+12,50.838,9.644142e+25,50.270,7990.0,58.991,100.288,93.861,73007549,0.331,20.024,4400.0,107.749
2,97.889,1.315554e+12,51.407,1.004435e+26,51.448,8852.0,61.879,111.703,101.342,77645623,0.329,19.987,4478.0,120.003


In [5]:
length=dataframe.shape[1]-1

In [6]:
length

13

In [7]:
# split into input (X) and output (Y) variables
X = dataframe.iloc[:,0:length]
y = dataframe['priceUSD']

In [8]:
X.head(3)

,difficulty30rsi,difficulty90var,fee_to_reward90rsiUSD,hashrate30var,median_transaction_fee7rocUSD,mining_profitability,price30smaUSD,price3wmaUSD,price7wmaUSD,sentinusd90emaUSD,size90trx,top100cap,transactionvalueUSD
0,97.889,1.168546e+12,49.775,9.266842e+25,36.980,7220.0,56.481,91.616,88.156,68391477,0.334,19.962,2592.0
1,97.889,1.243755e+12,50.838,9.644142e+25,50.270,7990.0,58.991,100.288,93.861,73007549,0.331,20.024,4400.0
2,97.889,1.315554e+12,51.407,1.004435e+26,51.448,8852.0,61.879,111.703,101.342,77645623,0.329,19.987,4478.0


In [9]:
y=np.ravel(y)

In [10]:
shape=X.shape[1]

In [11]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

In [12]:
estimators=[]

In [13]:
estimators.append(['mixmax',MinMaxScaler()])

In [14]:
estimators.append(['robust',RobustScaler()])

In [15]:
scale=Pipeline(estimators,verbose=True)

In [16]:
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing mixmax, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing robust, total=   0.0s


Pipeline(memory=None,
         steps=[('mixmax', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ['robust',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)]],
         verbose=True)

In [17]:
X_train=scale.transform(X_train)

In [18]:
X_test=scale.transform(X_test)

In [19]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [20]:
# define neural network model
def sequential_model(initializer='normal', activation='relu', neurons=350, NUM_FEATURES=shape):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, kernel_initializer=initializer))
    # Compile model
    adam=keras.optimizers.adam(lr=lr_schedule(0))
    model.compile(loss='logcosh', optimizer=adam, metrics=['mae'])
    return model

In [21]:
regressor=KerasRegressor(build_fn=sequential_model, batch_size=64, epochs=1000,verbose=1, shuffle=True )

In [22]:
results=cross_validate(regressor,X_train,y_train,
                       cv=5,
                       scoring=('r2', 'neg_mean_squared_error','neg_mean_absolute_error'), 
                       return_train_score=True, 
                       return_estimator=True)
Audio(sound_file,autoplay=True)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
1577/1577 [==============================] - 0s 107us/step - loss: 2999.0052 - mae: 2999.6982
Epoch 2/1000
1577/1577 [==============================] - 0s 54us/step - loss: 2975.4293 - mae: 2976.1228
Epoch 3/1000
1577/1577 [==============================] - 0s 54us/step - loss: 2857.2125 - mae: 2857.9058
Epoch 4/1000
1577/1577 [==============================] - 0s 54us/step - loss: 2477.3358 - mae: 2478.0286
Epoch 5/1000
1577/1577 [==============================] - 0s 53us/step - loss: 1685.1573 - mae: 1685.8503
Epoch 6/1000
1577/1577 [==============================] - 0s 49us/step - loss: 1230.6159 - mae: 1231.3091
Epoch 7/1000
1577/1577 [==============================] - 0s 57us/step - loss: 1056.6887 - mae: 1057.3817
Epoch 8/1000
1577/1577 [==============================] - 0s 48us/step - loss: 885.0562 - mae: 885.7477
Epoch 9/100

1577/1577 [==============================] - 0s 55us/step - loss: 90.4812 - mae: 91.1550
Epoch 74/1000
1577/1577 [==============================] - 0s 57us/step - loss: 96.4374 - mae: 97.1170
Epoch 75/1000
1577/1577 [==============================] - 0s 56us/step - loss: 92.8621 - mae: 93.5428
Epoch 76/1000
1577/1577 [==============================] - 0s 53us/step - loss: 106.3893 - mae: 107.0698
Epoch 77/1000
1577/1577 [==============================] - 0s 50us/step - loss: 92.6048 - mae: 93.2858
Epoch 78/1000
1577/1577 [==============================] - 0s 57us/step - loss: 90.6265 - mae: 91.3054
Epoch 79/1000
1577/1577 [==============================] - 0s 55us/step - loss: 92.3801 - mae: 93.0609
Epoch 80/1000
1577/1577 [==============================] - 0s 55us/step - loss: 86.9606 - mae: 87.6400
Epoch 81/1000
1577/1577 [==============================] - 0s 54us/step - loss: 85.9920 - mae: 86.6716
Epoch 82/1000
1577/1577 [==============================] - 0s 53us/step - loss: 84.84

1577/1577 [==============================] - 0s 57us/step - loss: 70.2467 - mae: 70.9220
Epoch 153/1000
1577/1577 [==============================] - 0s 53us/step - loss: 70.0545 - mae: 70.7282
Epoch 154/1000
1577/1577 [==============================] - 0s 54us/step - loss: 74.1297 - mae: 74.8072
Epoch 155/1000
1577/1577 [==============================] - 0s 57us/step - loss: 73.5002 - mae: 74.1706
Epoch 156/1000
1577/1577 [==============================] - 0s 59us/step - loss: 84.7624 - mae: 85.4409
Epoch 157/1000
1577/1577 [==============================] - 0s 56us/step - loss: 75.2561 - mae: 75.9347
Epoch 158/1000
1577/1577 [==============================] - 0s 54us/step - loss: 70.3780 - mae: 71.0546
Epoch 159/1000
1577/1577 [==============================] - 0s 54us/step - loss: 68.4860 - mae: 69.1641
Epoch 160/1000
1577/1577 [==============================] - 0s 56us/step - loss: 67.9311 - mae: 68.6028
Epoch 161/1000
1577/1577 [==============================] - 0s 53us/step - loss

1577/1577 [==============================] - 0s 57us/step - loss: 66.8601 - mae: 67.5354
Epoch 231/1000
1577/1577 [==============================] - 0s 54us/step - loss: 64.6697 - mae: 65.3384
Epoch 232/1000
1577/1577 [==============================] - 0s 54us/step - loss: 73.3769 - mae: 74.0474
Epoch 233/1000
1577/1577 [==============================] - 0s 54us/step - loss: 71.3009 - mae: 71.9774
Epoch 234/1000
1577/1577 [==============================] - 0s 56us/step - loss: 63.9654 - mae: 64.6330
Epoch 235/1000
1577/1577 [==============================] - 0s 55us/step - loss: 62.7594 - mae: 63.4308
Epoch 236/1000
1577/1577 [==============================] - 0s 54us/step - loss: 65.5526 - mae: 66.2254
Epoch 237/1000
1577/1577 [==============================] - 0s 51us/step - loss: 62.3991 - mae: 63.0679
Epoch 238/1000
1577/1577 [==============================] - 0s 56us/step - loss: 62.9440 - mae: 63.6123
Epoch 239/1000
1577/1577 [==============================] - 0s 58us/step - loss

1577/1577 [==============================] - 0s 57us/step - loss: 59.5853 - mae: 60.2498
Epoch 309/1000
1577/1577 [==============================] - 0s 50us/step - loss: 63.7888 - mae: 64.4593
Epoch 310/1000
1577/1577 [==============================] - 0s 48us/step - loss: 61.6335 - mae: 62.2978
Epoch 311/1000
1577/1577 [==============================] - 0s 56us/step - loss: 63.4219 - mae: 64.0938
Epoch 312/1000
1577/1577 [==============================] - 0s 55us/step - loss: 62.2883 - mae: 62.9560
Epoch 313/1000
1577/1577 [==============================] - 0s 54us/step - loss: 59.1370 - mae: 59.8054
Epoch 314/1000
1577/1577 [==============================] - 0s 54us/step - loss: 61.6408 - mae: 62.3127
Epoch 315/1000
1577/1577 [==============================] - 0s 49us/step - loss: 63.0972 - mae: 63.7712
Epoch 316/1000
1577/1577 [==============================] - 0s 59us/step - loss: 60.7137 - mae: 61.3859
Epoch 317/1000
1577/1577 [==============================] - 0s 54us/step - loss

...

1578/1578 [==============================] - 0s 61us/step - loss: 52.0948 - mae: 52.7610
Epoch 676/1000
1578/1578 [==============================] - 0s 50us/step - loss: 53.1549 - mae: 53.8237
Epoch 677/1000
1578/1578 [==============================] - 0s 49us/step - loss: 51.9525 - mae: 52.6190
Epoch 678/1000
1578/1578 [==============================] - 0s 52us/step - loss: 50.2235 - mae: 50.8907
Epoch 679/1000
1578/1578 [==============================] - 0s 51us/step - loss: 49.9853 - mae: 50.6524
Epoch 680/1000
1578/1578 [==============================] - 0s 51us/step - loss: 49.8484 - mae: 50.5139
Epoch 681/1000
1578/1578 [==============================] - 0s 51us/step - loss: 48.8307 - mae: 49.4949
Epoch 682/1000
1578/1578 [==============================] - 0s 51us/step - loss: 49.0696 - mae: 49.7289
Epoch 683/1000
1578/1578 [==============================] - 0s 48us/step - loss: 55.1589 - mae: 55.8328
Epoch 684/1000
1578/1578 [==============================] - 0s 52us/step - loss

1578/1578 [==============================] - 0s 52us/step - loss: 49.6289 - mae: 50.2888
Epoch 754/1000
1578/1578 [==============================] - 0s 47us/step - loss: 50.5418 - mae: 51.2116
Epoch 755/1000
1578/1578 [==============================] - 0s 50us/step - loss: 49.3856 - mae: 50.0582
Epoch 756/1000
1578/1578 [==============================] - 0s 48us/step - loss: 47.6545 - mae: 48.3246
Epoch 757/1000
1578/1578 [==============================] - 0s 48us/step - loss: 46.7657 - mae: 47.4336
Epoch 758/1000
1578/1578 [==============================] - 0s 49us/step - loss: 46.0026 - mae: 46.6626
Epoch 759/1000
1578/1578 [==============================] - 0s 50us/step - loss: 46.5027 - mae: 47.1664
Epoch 760/1000
1578/1578 [==============================] - 0s 48us/step - loss: 48.1627 - mae: 48.8235
Epoch 761/1000
1578/1578 [==============================] - 0s 49us/step - loss: 48.6271 - mae: 49.2922
Epoch 762/1000
1578/1578 [==============================] - 0s 49us/step - loss

1578/1578 [==============================] - 0s 54us/step - loss: 46.9936 - mae: 47.6583
Epoch 832/1000
1578/1578 [==============================] - 0s 49us/step - loss: 45.8624 - mae: 46.5283
Epoch 833/1000
1578/1578 [==============================] - 0s 51us/step - loss: 48.2942 - mae: 48.9630
Epoch 834/1000
1578/1578 [==============================] - 0s 52us/step - loss: 46.7429 - mae: 47.4030
Epoch 835/1000
1578/1578 [==============================] - 0s 50us/step - loss: 46.7456 - mae: 47.4138
Epoch 836/1000
1578/1578 [==============================] - 0s 49us/step - loss: 47.9614 - mae: 48.6244
Epoch 837/1000
1578/1578 [==============================] - 0s 47us/step - loss: 48.0371 - mae: 48.7085
Epoch 838/1000
1578/1578 [==============================] - 0s 48us/step - loss: 52.9781 - mae: 53.6460
Epoch 839/1000
1578/1578 [==============================] - 0s 49us/step - loss: 50.8679 - mae: 51.5327
Epoch 840/1000
1578/1578 [==============================] - 0s 50us/step - loss

1578/1578 [==============================] - 0s 52us/step - loss: 47.3491 - mae: 48.0096
Epoch 910/1000
1578/1578 [==============================] - 0s 51us/step - loss: 71.1121 - mae: 71.7839
Epoch 911/1000
1578/1578 [==============================] - 0s 49us/step - loss: 54.1972 - mae: 54.8664
Epoch 912/1000
1578/1578 [==============================] - 0s 52us/step - loss: 50.1875 - mae: 50.8545
Epoch 913/1000
1578/1578 [==============================] - 0s 52us/step - loss: 45.4717 - mae: 46.1341
Epoch 914/1000
1578/1578 [==============================] - 0s 51us/step - loss: 45.4875 - mae: 46.1469
Epoch 915/1000
1578/1578 [==============================] - 0s 49us/step - loss: 46.6428 - mae: 47.3071
Epoch 916/1000
1578/1578 [==============================] - 0s 48us/step - loss: 45.7018 - mae: 46.3664
Epoch 917/1000
1578/1578 [==============================] - 0s 50us/step - loss: 45.5016 - mae: 46.1687
Epoch 918/1000
1578/1578 [==============================] - 0s 48us/step - loss

1578/1578 [==============================] - 0s 54us/step - loss: 47.4785 - mae: 48.1479
Epoch 988/1000
1578/1578 [==============================] - 0s 55us/step - loss: 46.3429 - mae: 47.0099
Epoch 989/1000
1578/1578 [==============================] - 0s 51us/step - loss: 44.7422 - mae: 45.4061
Epoch 990/1000
1578/1578 [==============================] - 0s 53us/step - loss: 45.2614 - mae: 45.9264
Epoch 991/1000
1578/1578 [==============================] - 0s 54us/step - loss: 45.2372 - mae: 45.9082
Epoch 992/1000
1578/1578 [==============================] - 0s 52us/step - loss: 42.5441 - mae: 43.2015
Epoch 993/1000
1578/1578 [==============================] - 0s 55us/step - loss: 46.1364 - mae: 46.7995
Epoch 994/1000
1578/1578 [==============================] - 0s 53us/step - loss: 45.2192 - mae: 45.8807
Epoch 995/1000
1578/1578 [==============================] - 0s 57us/step - loss: 49.9870 - mae: 50.6503
Epoch 996/1000
1578/1578 [==============================] - 0s 50us/step - loss

In [23]:
results['train_r2']

array([0.99920982, 0.99920704, 0.99913821, 0.9992003 , 0.9992462 ])

In [24]:
results['test_r2']

array([0.99904828, 0.99897208, 0.9990181 , 0.99893092, 0.99880937])

In [25]:
results['estimator']

(<keras.wrappers.scikit_learn.KerasRegressor at 0x7f959f002630>,
 <keras.wrappers.scikit_learn.KerasRegressor at 0x7f957c38a0b8>)

In [26]:
# create directory for models
dir = 'stacked_ann'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

In [26]:
# save models
n_members = len(results['estimator'])
for i in range(n_members):
    # save model
    filename = 'stacked_ann/stacked_ANN_reg_seven_' + str(i + 1) + '.h5'
    results['estimator'][i].model.save(filename)
    print('>Saved %s' % filename)

>Saved stacked_ann/stacked_ANN_pca_reg1.h5
>Saved stacked_ann/stacked_ANN_pca_reg2.h5
>Saved stacked_ann/stacked_ANN_pca_reg3.h5
>Saved stacked_ann/stacked_ANN_pca_reg4.h5
>Saved stacked_ann/stacked_ANN_pca_reg5.h5


In [32]:
# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'stacked_ann/stacked_ANN_reg_seven_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

In [41]:
# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer.name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden1 = Dense(128, activation='relu')(merge)
    #hidden2= Dense(350,activation='relu')(hidden1)
    output = Dense(1, activation='relu')(hidden1)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
    plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    adam=keras.optimizers.adam(lr=lr_schedule(0))
    model.compile(loss='logcosh', optimizer='adam', metrics=['mae'])
    return model

In [38]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# fit model
	model.fit(X, inputy, batch_size=64, epochs=1000, verbose=1)

In [35]:
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [36]:
# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded stacked_ann/stacked_ANN_pca_reg1.h5
>loaded stacked_ann/stacked_ANN_pca_reg2.h5
>loaded stacked_ann/stacked_ANN_pca_reg3.h5
>loaded stacked_ann/stacked_ANN_pca_reg4.h5
>loaded stacked_ann/stacked_ANN_pca_reg5.h5
Loaded 5 models


In [39]:
# define ensemble model
stacked_model = define_stacked_model(members)


In [42]:
# fit stacked model on test dataset
fit_stacked_model(stacked_model,X_test, y_test)
Audio(sound_file,autoplay=True)

Epoch 1/1000
147/147 [==============================] - 0s 102us/step - loss: 60.2215 - mae: 60.8940
Epoch 2/1000
147/147 [==============================] - 0s 112us/step - loss: 60.7276 - mae: 61.3997
Epoch 3/1000
147/147 [==============================] - 0s 92us/step - loss: 59.8062 - mae: 60.4907
Epoch 4/1000
147/147 [==============================] - 0s 98us/step - loss: 61.5074 - mae: 62.1922
Epoch 5/1000
147/147 [==============================] - 0s 128us/step - loss: 61.1156 - mae: 61.7947
Epoch 6/1000
147/147 [==============================] - 0s 96us/step - loss: 61.6677 - mae: 62.3469
Epoch 7/1000
147/147 [==============================] - 0s 121us/step - loss: 60.3973 - mae: 61.0781
Epoch 8/1000
147/147 [==============================] - 0s 93us/step - loss: 60.1971 - mae: 60.8775
Epoch 9/1000
147/147 [==============================] - 0s 105us/step - loss: 61.2137 - mae: 61.8851
Epoch 10/1000
147/147 [==============================] - 0s 99us/step - loss: 60.6132 - mae: 61

147/147 [==============================] - 0s 113us/step - loss: 59.7579 - mae: 60.4385
Epoch 161/1000
147/147 [==============================] - 0s 113us/step - loss: 59.1546 - mae: 59.8375
Epoch 162/1000
147/147 [==============================] - 0s 131us/step - loss: 60.3855 - mae: 61.0669
Epoch 163/1000
147/147 [==============================] - 0s 104us/step - loss: 58.9243 - mae: 59.6022
Epoch 164/1000
147/147 [==============================] - 0s 100us/step - loss: 59.5817 - mae: 60.2647
Epoch 165/1000
147/147 [==============================] - 0s 97us/step - loss: 61.5742 - mae: 62.2577
Epoch 166/1000
147/147 [==============================] - 0s 103us/step - loss: 61.1008 - mae: 61.7860
Epoch 167/1000
147/147 [==============================] - 0s 105us/step - loss: 60.3252 - mae: 61.0069
Epoch 168/1000
147/147 [==============================] - 0s 117us/step - loss: 63.0453 - mae: 63.7300
Epoch 169/1000
147/147 [==============================] - 0s 108us/step - loss: 60.1206 -

147/147 [==============================] - 0s 121us/step - loss: 57.6169 - mae: 58.2758
Epoch 319/1000
147/147 [==============================] - 0s 147us/step - loss: 57.7957 - mae: 58.4744
Epoch 320/1000
147/147 [==============================] - 0s 115us/step - loss: 58.1420 - mae: 58.8242
Epoch 321/1000
147/147 [==============================] - 0s 122us/step - loss: 57.7375 - mae: 58.4063
Epoch 322/1000
147/147 [==============================] - 0s 107us/step - loss: 57.7562 - mae: 58.4431
Epoch 323/1000
147/147 [==============================] - 0s 114us/step - loss: 58.3729 - mae: 59.0496
Epoch 324/1000
147/147 [==============================] - 0s 89us/step - loss: 60.4183 - mae: 61.1056
Epoch 325/1000
147/147 [==============================] - 0s 120us/step - loss: 60.1199 - mae: 60.8073
Epoch 326/1000
147/147 [==============================] - 0s 117us/step - loss: 59.4944 - mae: 60.1758
Epoch 327/1000
147/147 [==============================] - 0s 104us/step - loss: 58.5832 -

147/147 [==============================] - 0s 139us/step - loss: 57.2641 - mae: 57.9381
Epoch 477/1000
147/147 [==============================] - 0s 131us/step - loss: 57.3021 - mae: 57.9724
Epoch 478/1000
147/147 [==============================] - 0s 155us/step - loss: 57.0482 - mae: 57.7003
Epoch 479/1000
147/147 [==============================] - 0s 125us/step - loss: 56.9287 - mae: 57.6030
Epoch 480/1000
147/147 [==============================] - 0s 102us/step - loss: 56.8085 - mae: 57.4765
Epoch 481/1000
147/147 [==============================] - 0s 116us/step - loss: 56.7836 - mae: 57.4561
Epoch 482/1000
147/147 [==============================] - 0s 106us/step - loss: 56.6713 - mae: 57.3330
Epoch 483/1000
147/147 [==============================] - 0s 107us/step - loss: 56.5686 - mae: 57.2241
Epoch 484/1000
147/147 [==============================] - 0s 119us/step - loss: 56.8074 - mae: 57.4722
Epoch 485/1000
147/147 [==============================] - 0s 138us/step - loss: 57.8151 

147/147 [==============================] - 0s 104us/step - loss: 56.2477 - mae: 56.9177
Epoch 635/1000
147/147 [==============================] - 0s 131us/step - loss: 56.1723 - mae: 56.8465
Epoch 636/1000
147/147 [==============================] - 0s 109us/step - loss: 56.0586 - mae: 56.7330
Epoch 637/1000
147/147 [==============================] - 0s 106us/step - loss: 56.5718 - mae: 57.2515
Epoch 638/1000
147/147 [==============================] - 0s 106us/step - loss: 56.3089 - mae: 56.9771
Epoch 639/1000
147/147 [==============================] - 0s 108us/step - loss: 56.4320 - mae: 57.1025
Epoch 640/1000
147/147 [==============================] - 0s 109us/step - loss: 56.8481 - mae: 57.5197
Epoch 641/1000
147/147 [==============================] - 0s 108us/step - loss: 56.1022 - mae: 56.7662
Epoch 642/1000
147/147 [==============================] - 0s 90us/step - loss: 58.1696 - mae: 58.8532
Epoch 643/1000
147/147 [==============================] - 0s 91us/step - loss: 57.3833 - 

147/147 [==============================] - 0s 134us/step - loss: 55.9953 - mae: 56.6753
Epoch 793/1000
147/147 [==============================] - 0s 129us/step - loss: 55.7353 - mae: 56.4068
Epoch 794/1000
147/147 [==============================] - 0s 132us/step - loss: 55.6772 - mae: 56.3455
Epoch 795/1000
147/147 [==============================] - 0s 120us/step - loss: 55.7786 - mae: 56.4525
Epoch 796/1000
147/147 [==============================] - 0s 117us/step - loss: 55.6959 - mae: 56.3741
Epoch 797/1000
147/147 [==============================] - 0s 97us/step - loss: 55.9316 - mae: 56.5939
Epoch 798/1000
147/147 [==============================] - 0s 91us/step - loss: 55.6165 - mae: 56.2919
Epoch 799/1000
147/147 [==============================] - 0s 102us/step - loss: 55.3885 - mae: 56.0561
Epoch 800/1000
147/147 [==============================] - 0s 101us/step - loss: 55.7257 - mae: 56.4043
Epoch 801/1000
147/147 [==============================] - 0s 112us/step - loss: 55.9732 - 

147/147 [==============================] - 0s 95us/step - loss: 56.6301 - mae: 57.3159
Epoch 951/1000
147/147 [==============================] - 0s 131us/step - loss: 55.7058 - mae: 56.3806
Epoch 952/1000
147/147 [==============================] - 0s 78us/step - loss: 56.7200 - mae: 57.4023
Epoch 953/1000
147/147 [==============================] - 0s 96us/step - loss: 55.4208 - mae: 56.1027
Epoch 954/1000
147/147 [==============================] - 0s 107us/step - loss: 55.2883 - mae: 55.9741
Epoch 955/1000
147/147 [==============================] - 0s 108us/step - loss: 55.0366 - mae: 55.7188
Epoch 956/1000
147/147 [==============================] - 0s 88us/step - loss: 55.0458 - mae: 55.7180
Epoch 957/1000
147/147 [==============================] - 0s 72us/step - loss: 54.8829 - mae: 55.5596
Epoch 958/1000
147/147 [==============================] - 0s 97us/step - loss: 55.2247 - mae: 55.9004
Epoch 959/1000
147/147 [==============================] - 0s 122us/step - loss: 55.9235 - mae:

In [43]:
# make predictions
y_pred = predict_stacked_model(stacked_model, X_test)

In [44]:
y_pred=np.ravel(y_pred)

In [45]:
pd.DataFrame(zip(y_test,y_pred),columns=['y_test','y_pred'])

,y_test,y_pred
0,283.365,320.175781
1,383.996,395.632904
2,312.341,387.003082
3,838.570,550.610718
4,604.807,600.659729
5,96.796,155.191254
6,415.876,510.364532
7,347.500,397.292908
8,234.532,318.166656
9,345.878,385.888184


In [46]:
r2(np.ravel(y_test),y_pred)

0.8315096931741556

In [47]:
mae(y_test,y_pred) #mae

55.43160097623033

In [48]:
mse(y_test,y_pred)

9116.019444141113

In [49]:
mean_absolute_percentage_error(y_test,np.ravel(y_pred)) #mape

17.687510672433152

In [41]:
np.sqrt(mse(y_test,y_pred))#rmse

8.006364353849213

In [42]:
stacked_model.save('trained_models/stacked_ANN_reg_seven_new.h5')

In [138]:
stacked_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_19_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
dense_22_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
dense_25_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
dense_28_input (InputLayer)     (None, 20)           0                                            
____________________________________________________________________________________________

In [139]:
model=load_model('trained_models/stacked_ANN_reg_seven_new.h5',compile=False)

In [113]:
y_pred = predict_stacked_model(model, X_test)

In [140]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_19_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
dense_22_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
dense_25_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
dense_28_input (InputLayer)     (None, 20)           0                                            
____________________________________________________________________________________________

In [115]:
predictions=pd.DataFrame(zip(y_test,np.ravel(y_pred)),columns=['y_test','y_pred'])

In [116]:
predictions

,y_test,y_pred
0,583.850,588.796021
1,615.988,667.171143
2,234.619,247.882675
3,5260.000,5420.366699
4,252.788,255.497742
5,379.647,389.013367
6,644.487,631.135864
7,7260.000,7658.349121
8,312.724,327.399933
9,3780.000,3715.978760


In [117]:
predictions.to_csv('predictions/stacked_ANN_reg_thirty.csv')